In [8]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import pydicom
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler

torch.backends.cudnn.benchmark = True
# Set device for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")



Using cuda device


In [9]:
# Custom Weighted Logarithmic Loss
class WeightedLogLoss(nn.Module):
    def __init__(self, weights=None):
        super().__init__()
        self.epsilon = 1e-15
        self.weights = torch.tensor(weights, dtype=torch.float32).to(device) if weights else None

    def forward(self, preds, targets):
        preds = torch.clamp(preds, min=self.epsilon, max=1 - self.epsilon)
        log_loss = - (targets * torch.log(preds) + (1 - targets) * torch.log(1 - preds))
        
        if self.weights is not None:
            log_loss *= self.weights
        return log_loss.mean()


In [10]:
# ROC Curve Visualization
def plot_roc_curves(y_true, y_pred, classes):
    plt.figure(figsize=(10, 8))
    for i, class_name in enumerate(classes):
        fpr, tpr, _ = roc_curve(y_true[:, i], y_pred[:, i])
        plt.plot(fpr, tpr, label=f"ROC curve ({class_name})")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curves")
    plt.legend(loc="lower right")
    plt.show()

In [11]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1))
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.title("Confusion Matrix")
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()

In [12]:
# Evaluate Model and Plot Confusion Matrix
def evaluate_and_plot(model, data_loader, criterion, class_names):
    model.eval()
    y_true, y_pred = [], []
    val_loss = 0

    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = torch.sigmoid(model(inputs))
            val_loss += criterion(outputs, labels).item()

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(outputs.cpu().numpy())

    y_true = np.array(y_true)
    y_pred = (np.array(y_pred) > 0.5).astype(int)

    # Calculate Evaluation Metrics
    accuracy = accuracy_score(y_true.argmax(axis=1), y_pred.argmax(axis=1))
    sensitivity = recall_score(y_true.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
    specificity = precision_score(y_true.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
    roc_auc = roc_auc_score(y_true, y_pred, average='weighted', multi_class='ovr')

    print(f"\nValidation Loss: {val_loss / len(data_loader):.4f}")
    print(f"Accuracy: {accuracy:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}, ROC AUC: {roc_auc:.4f}")

    # Plot Confusion Matrix
    plot_confusion_matrix(y_true, y_pred, class_names)
    plot_roc_curves(y_true, y_pred, class_names)

    return accuracy, sensitivity, specificity, roc_auc


In [13]:
class_names = ["epidural", "intraparenchymal", "intraventricular", "subarachnoid", "subdural", "any"]

# Evaluate and plot confusion matrix
criterion = WeightedLogLoss(weights=weights)
evaluate_and_plot(model, val_loader, criterion, class_names)

NameError: name 'weights' is not defined

In [14]:
# Import Libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
import pydicom
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score


In [15]:
import os
import pydicom
from PIL import Image
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

# Convert DICOM to PNG
def convert_dicom_to_png_worker(args):
    dicom_path, output_dir = args
    try:
        dcm = pydicom.read_file(dicom_path)
        img = dcm.pixel_array.astype(np.float32)

        # Normalize image to [0, 255]
        img_min = img.min()
        img_max = img.max()
        img = ((img - img_min) / (img_max - img_min) * 255).astype(np.uint8)

        # Save as PNG
        img_id = os.path.basename(dicom_path).replace(".dcm", ".png")
        output_file = os.path.join(output_dir, img_id)
        Image.fromarray(img).save(output_file)
    except Exception as e:
        print(f"Error processing {dicom_path}: {e}")


# Batch Convert DICOM to PNG with Multi-Processing
def batch_convert_to_png_mp(dicom_dir, output_dir, num_workers=72):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    dicom_files = [
        os.path.join(dicom_dir, file)
        for file in os.listdir(dicom_dir)
        if file.endswith(".dcm")
    ]

    # Use all available CPUs if not specified
    num_workers = num_workers or cpu_count()

    print(f"Starting Multi-Processing with {num_workers} workers...")
    with Pool(num_workers) as pool:
        list(tqdm(pool.imap_unordered(
            convert_dicom_to_png_worker, [(file, output_dir) for file in dicom_files]
        ), total=len(dicom_files), desc="Converting DICOM to PNG"))


In [16]:
# Preprocess CSV Labels
def preprocess(path):
    df = pd.read_csv(path)
    df[['ImageID', 'Subtype']] = df['ID'].str.rsplit('_', n=1, expand=True)
    df = df.groupby(['ImageID', 'Subtype'])['Label'].max().unstack(fill_value=0).reset_index()
    for col in df.columns[1:]:
        df[col] = df[col].astype('float32')
    return df


In [17]:
# Custom Dataset for PNG Images
class PngDataset(Dataset):
    def __init__(self, img_dir, df, transform=None):
        self.img_dir = img_dir
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, f'{img_id}.png')

        # Load PNG image
        img = Image.open(img_path).convert('L')  # Grayscale

        if self.transform:
            img = self.transform(img)

        label = torch.tensor(self.df.iloc[idx, 1:].values.astype('float32'))
        return img, label


In [18]:
# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, n_epochs=5):
    best_roc_auc = 0.0
    for epoch in range(n_epochs):
        print(f"\nEpoch {epoch+1}/{n_epochs}")
        model.train()
        train_loss = 0

        for inputs, labels in tqdm(train_loader, desc="Training"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = torch.sigmoid(model(inputs))
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        val_loss, accuracy, sensitivity, specificity, roc_auc = evaluate_model(model, val_loader, criterion)

        print(f"\nTrain Loss: {train_loss / len(train_loader):.4f}, Val Loss: {val_loss:.4f}")
        print(f"Accuracy: {accuracy:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}, ROC AUC: {roc_auc:.4f}")

        # Save the best model
        if roc_auc > best_roc_auc:
            print(f"Saving Best Model with ROC AUC: {roc_auc:.4f}")
            torch.save(model.state_dict(), "best_model.pth")
            best_roc_auc = roc_auc

# Evaluation Function
def evaluate_model(model, data_loader, criterion):
    model.eval()
    y_true, y_pred = [], []
    val_loss = 0

    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = torch.sigmoid(model(inputs))
            val_loss += criterion(outputs, labels).item()

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(outputs.cpu().numpy())

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    accuracy = accuracy_score(y_true.argmax(axis=1), y_pred.argmax(axis=1))
    sensitivity = recall_score(y_true.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
    specificity = precision_score(y_true.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
    roc_auc = roc_auc_score(y_true, y_pred, average='weighted', multi_class='ovr')

    return val_loss / len(data_loader), accuracy, sensitivity, specificity, roc_auc


In [19]:
# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model Definition
model = models.resnet34(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 6)
model.to(device)

# Optimizer and Loss Function
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-5)


/home2/s439765/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home2/s439765/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Define Paths
TRAIN_DICOM_DIR = './rsna-intracranial-hemorrhage-detection/stage_2_train'
TRAIN_PNG_DIR = './rsna-intracranial-hemorrhage-detection/train_png'
CSV_PATH = './rsna-intracranial-hemorrhage-detection/stage_2_train.csv'

# Convert Images
#batch_convert_to_png_mp(TRAIN_DICOM_DIR, TRAIN_PNG_DIR)

# Preprocess CSV
df = preprocess(CSV_PATH)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Define Data Loaders
transform = transforms.Compose([
    transforms.Resize((128, 128), antialias=True),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_dataset = PngDataset(TRAIN_PNG_DIR, train_df, transform=transform)
val_dataset = PngDataset(TRAIN_PNG_DIR, val_df, transform=transform)

train_loader = DataLoader(
    train_dataset, batch_size=128, shuffle=True, num_workers=16, pin_memory=True
)

val_loader = DataLoader(
    val_dataset, batch_size=128, shuffle=False, num_workers=16, pin_memory=True
)

# Train the Model
train_model(model, train_loader, val_loader, criterion, optimizer, n_epochs=1)



Epoch 1/1


Training:  74%|███████▍  | 3505/4706 [1:29:41<12:10,  1.64it/s]  